In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM")

model = AutoModelForMaskedLM.from_pretrained("DeepChem/ChemBERTa-77M-MLM")

/home/pj20/miniconda3/envs/txgnn_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import json 
from tqdm import tqdm

with open('./idx_map.json') as f:
    idx_map = json.load(f)

molecule_idx2cid = {value: key for key, value in idx_map['molecule'].items()}

In [1]:
import pickle

with open('./cid2emb_new.pkl', 'rb') as f:
    cid2emb = pickle.load(f)

In [4]:
cid2emb.keys()

dict_keys([73219, 23551636, None, 12643743, 7420, 8609, 17435, 3569560, 6001, 3385, 2763750, 142523, 34468, 12671690, 143636537, 85768913, 131411056, 5665, 6720, 69332716, 49837, 18768052, 136506065, 58137106, 71721420, 67246787, 83303, 11546947, 11535547, 418247, 154108049, 129671452, 3520, 3851247, 130026107, 55255285, 2777208, 134558090, 12089, 26794240, 516910, 13755, 23269040, 6321253, 12346975, 113033, 2724, 44565819, 11181, 71450668, 10444765, 135418174, 442348, 65985700, 19658, 5161, 12582, 15513, 104810, 8795, 145915578, 7654, 57391458, 3016408, 541384, 59893068, 19166, 129654318, 6417, 7276, 8246, 23394, 6713368, 13331108, 51576, 72823952, 20629, 133621, 45486169, 11039, 3105, 7924, 102115685, 44570629, 154536664, 74937, 6433082, 71771, 12993143, 122163231, 144152010, 140147, 91776, 237332, 10452, 815, 14994, 65067, 61799, 72187525, 57261105, 644213, 90041889, 165444336, 70686983, 15289, 219602, 17779323, 54526114, 54502555, 3356, 12328, 10701806, 20782688, 54584753, 89094825

In [27]:
with open('./smile2cid.json', 'r') as f:
    smile2cid = json.load(f)

cid2smile = {v: k for k, v in smile2cid.items()}

In [ ]:
from tqdm import tqdm
import pubchempy as pcp
from collections import defaultdict
import time
import concurrent.futures
import pickle


# cid2emb = defaultdict(dict)

def get_embeddings(cid):
    for i in range(5):
        try:
            # Define the SMILES string
            p = pcp.get_properties('IsomericSMILES', cid, 'cid')
            smiles_string = p[0]['IsomericSMILES']

            # Tokenize the SMILES string
            tokens = tokenizer(smiles_string, return_tensors="pt")
            if len(tokens['input_ids'][0]) > 500:
                tokens = tokenizer(smiles_string[:500], return_tensors="pt")

            # Pass the tokens through the model
            outputs = model(**tokens)
            # Extract the embeddings
            embeddings = outputs.logits
            # Calculate the mean embedding
            mean_embedding = embeddings.mean(dim=1)

            return cid, smiles_string, mean_embedding[0].detach().numpy()

        except pcp.PubChemHTTPError:
            # If there is a PubChemHTTPError, wait for 1 second and try again
            time.sleep(1)
    return cid, None, None

def get_embeddings_1(cid):
        smiles_string = cid2smile[cid]

        # Tokenize the SMILES string
        tokens = tokenizer(smiles_string, return_tensors="pt")
        if len(tokens['input_ids'][0]) > 500:
            tokens = tokenizer(smiles_string[:500], return_tensors="pt")

        # Pass the tokens through the model
        outputs = model(**tokens)
        # Extract the embeddings
        embeddings = outputs.logits
        # Calculate the mean embedding
        mean_embedding = embeddings.mean(dim=1)

        return cid, smiles_string, mean_embedding[0].detach().numpy()


# Use a ThreadPoolExecutor to parallelize the processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    cids = list(molecule_idx2cid.values())
    cids = [cid.replace('.0', '') for cid in cids]
    for cid, smiles_string, mean_embedding in tqdm(executor.map(get_embeddings, cids), total=len(cids)):
        if cid in cid2emb:
            continue
        if smiles_string is not None and mean_embedding is not None:
            cid2emb[cid]['smiles'] = smiles_string
            cid2emb[cid]['emb'] = mean_embedding
        
        if len(cid2emb) % 10000 == 0 and len(cid2emb) > 0:
            with open('./cid2emb_new.pkl', 'wb') as f:
                pickle.dump(cid2emb, f)


In [10]:
cids = list(molecule_idx2cid.values())
cids = [cid.replace('.0', '') for cid in cids]

In [9]:
p = pcp.get_properties('IsomericSMILES', "10", 'cid')
smiles_string = p[0]['IsomericSMILES']

# Tokenize the SMILES string
tokens = tokenizer(smiles_string, return_tensors="pt")
if len(tokens['input_ids'][0]) > 500:
    tokens = tokenizer(smiles_string[:500], return_tensors="pt")

# Pass the tokens through the model
outputs = model(**tokens)
# Extract the embeddings
embeddings = outputs.logits
# Calculate the mean embedding
mean_embedding = embeddings.mean(dim=1)

In [11]:
len(molecule_idx2cid)

88811